# 객체

::: {.hidden}

## 참고 문헌

- Pro Git, Git Internels, Git Objects
  - chrome-extension://oemmndcbldboiebfnladdacbdfmadadm/file:///Users/joelkim/Work/study/study_cs/book/DevOps/Progit/progit.pdf#%5B%7B%22num%22%3A2221%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2C0%2C510.57%2Cnull%5D
- Mastering Git, Ch10 Keeping History Clean, An introduction to Git internals
  - chrome-extension://oemmndcbldboiebfnladdacbdfmadadm/file:///Users/joelkim/Work/study/study_cs/book/DevOps/Narebski/2024%20-%20Narebski%20-%20Mastering%20Git,%202nd%20Edition.pdf#p263

:::

## 객체

- git의 대부분의 정보는 객체(object) 형태로 저장
- `commit` 명령으로 레포지토리에 정식 저장된 정보 뿐 아니라 `add` 명령으로 스테이지에 저장된 파일도 객체로 저장됨

### 객체로 저장되지 않는 정보

- 브랜치 정보는 객체가 아닌 참조(ref) 정보로 저장됨

## 해시값

- 모든 객체는 이름(아이디)으로 sha1 해시값 40글자를 가짐
- 객체의 유형에 따라 해시 계산 방법이 다름
  - 파일내용을 나타내는 blob 객체의 해시값은 파일의 내용만(파일 이름도 사용하지 않음!) 사용하여 계산하므로 파일 내용이 같으면 해시값도 같음
  - 디렉토리 내용을 나타내는 blob 객체의 해시값은 해당 디렉토리에 있는 파일 이름과 해시값(또는 서브디렉토리 이름 정보 사용하여 계산하므로 파일 내용이 같으면 해시값도 같음
  - 커밋을 나타내는 commit 객체의 해시값은 작성자 정보, 작성 시간 정보 등을 통합하여 계산하므로 커밋 파일의 내용이 같아도 해시값이 달라질 수 있음

## 객체 저장 위치

- 객체는 `.git/objects` 디렉토리 아래에 서브디렉토리로 나뉘어서 파일형태로 저장
- 40글자 sha1 해시값 중 첫 2글자를 디렉토리 이름으로 하고 나머지 38글자를 파일이름으로 저장
- 추가 디렉토리
  - `.git/objects/info`
  - `.git/objects/pack`

### 예시

- 샘플 레포지토리 생성

    ```bash
    $ mkdir test
    $ cd test
    $ git init
    Initialized empty Git repository in test/.git/
    $ echo "file1 line1" > file1.txt
    $ echo "file2 line1" > file2.txt
    $ mkdir sub
    $ echo "file3 line1" > sub/file3.txt
    $ git add .
    $ git commit -m "test"
    [main (root-commit) 3854a02] test
    3 files changed, 3 insertions(+)
    create mode 100644 file1.txt
    create mode 100644 file2.txt
    create mode 100644 sub/file3.txt
    ```

- 객체 목록 출력

    ```bash
    $ find .git/objects -type f
    .git/objects/0b/11cfca50e35a4865e8505f1a108bd23a3f9401
    .git/objects/bb/e651d471cd00fc3add5bdac5c6a41619555f7c
    .git/objects/42/0b1e064cb3f5a3ae63c99e1cc8ef46c8f004c9
    .git/objects/5c/07b4420198fbb25aabbba53aaa908ca0fb9dd5
    .git/objects/e6/782efee2e87f68c0d6d87d8bd2a1031dc21f45
    .git/objects/4a/8d995103b41f561e547ce3b32e1983d424da22
    ```

## 객체의 저장 형식

- zlib으로 압축된 바이너리 파일
- 객체를 hex 에디터로 보면 앞부분이 zlib DEFLATE 형식(0x78 0x9C)으로 시작함

## 객체의 종류

- blob 객체: 한 순간의 하나의 파일의 스냅샷 정보
- tree 객체: 한 순간의 하나의 디렉토리의 파일/서브디렉토리 정보
- commit 객체: 하나의 커밋 정보
- tag 객체: 태그 정보

## 객체를 보는 명령

- 저장소 전체의 객체 목록을 보는 법

    ```bash
    git cat-file --batch-check --batch-all-objects --unordered
    ```

- 객체 하나의 유형을 보는 법

    ```bash
    git cat-file -t 해시값
    ```

- 객체 하나의 파싱 정보를 보는 법

    ```bash
    git cat-file -p 해시값
    ```

- 객체의 내용 정보를 원 바이트열 그대로 보고 싶을 때는 zlib 압축을 풀 수 있는 도구가 있어야 한다.
  - 예를 들어 "qpdf" 애플리케이션을 설치하면 `zlib-flate -uncompress` 명령으로 객체 내용을 볼 수 있다.
  - 단, 이 경우에는 바이너리 바이트열을 그대로 보여주므로 콘솔에서 문자로 잘 표시되지 않을 수 있다. 


### 예시

```bash
$ git cat-file --batch-check --batch-all-objects --unordered
0b11cfca50e35a4865e8505f1a108bd23a3f9401 blob 12
420b1e064cb3f5a3ae63c99e1cc8ef46c8f004c9 commit 171
4a8d995103b41f561e547ce3b32e1983d424da22 blob 12
5c07b4420198fbb25aabbba53aaa908ca0fb9dd5 blob 12
bbe651d471cd00fc3add5bdac5c6a41619555f7c tree 104
e6782efee2e87f68c0d6d87d8bd2a1031dc21f45 tree 37

$ git cat-file -t 0b11cfca50e35a4865e8505f1a108bd23a3f9401
blob

$ git cat-file -p 0b11cfca50e35a4865e8505f1a108bd23a3f9401
file1 line1

$ zlib-flate -uncompress < ".git/objects/0b/11cfca50e35a4865e8505f1a108bd23a3f9401"
blob 12file1 line1
```

## blob 객체

### blob 객체의 내용

- 파일의 내용(바이트 스트림)만 저장
- 커밋이 아닌 스테이징 시점에 생성되고 커밋되면 tree 객체에서 참조하게 됨
- 파일명, 모드, 권한, 타임스탬프 등의 메타데이터 포함하지 않음
- 파일명과 모드는 blob 객체가 아닌 tree 객체에 저장됨

### blob 객체의 해시값

- "blob <바이트길이>\0<데이터바이트>" 형식의 바이트열을 sha1 계산


### 예시

```bash
$ printf "file1 line1\n" | git hash-object -t blob --stdin
0b11cfca50e35a4865e8505f1a108bd23a3f9401
$ printf "blob 12\0file1 line1\n" | sha1sum
0b11cfca50e35a4865e8505f1a108bd23a3f9401  -
```

### dangling blob

- 파일이 스테이징되어 blob 객체가 생성되었는데 커밋되지 않고 unstage가 되어도 해당 파일의 blob 객체는 남아있음
- tree 객체에서 참조하지 않는 blob 객체를 dangling blob 이라고 함

## tree 객체

- tree 객체는 하나의 디렉토리(서브디렉토리 미포함)에 포함된 파일과 서브디렉토리에 대한 해시값 정보
- 서브디렉토리가 존재하면 그에 해당하는 tree 객체가 추가적으로 존재


### tree 객체의 내용

- tree 객체의 내용은 엔트리 열로 구성됨
- 엔트리(entry)
  - 하나의 파일 또는 서브디렉토리를 대표하는 정보
  - 하나의 엔트리 내용은 "`<mode> <파일명>\0<해시값>`"
- 엔트리 열
  - 엔트리 정보를 이어붙인 "`<entry><entry><entry>...`" 형식
- 하나의 tree 객체 내용은 "`tree <크기>\0<엔트리 열>`" 형식

### tree 객체의 해시값

- tree 객체의 내용에 대해 SHA-1을 계산

### 예시

```bash
$ git cat-file -p bbe651d471cd00fc3add5bdac5c6a41619555f7c
100644 blob 0b11cfca50e35a4865e8505f1a108bd23a3f9401    file1.txt
100644 blob 4a8d995103b41f561e547ce3b32e1983d424da22    file2.txt
040000 tree e6782efee2e87f68c0d6d87d8bd2a1031dc21f45    sub

$ git cat-file -p e6782efee2e87f68c0d6d87d8bd2a1031dc21f45
100644 blob 5c07b4420198fbb25aabbba53aaa908ca0fb9dd5    file3.txt

$ zlib-flate -uncompress < ".git/objects/bb/e651d471cd00fc3add5bdac5c6a41619555f7c" 
tree 104100644 file1.txt
                        ��P�ZHe�P_��:?�100644 file2.txtJ��Q�VT|�.��$�"40000 sub�x.���h���}�ҡ�E%      
```

- 콘솔에서 정상 표시되지 않는 바이트열을 보고 싶으면 다음 파이썬 코드를 사용한다.

```python

# get_tree_object_content.py

import zlib
import sys
from pathlib import Path


def get_tree_object_content(data: bytes) -> bytes:
    output = bytearray()

    # 기존 헤더 ("tree <size>\x00") 그대로 복사
    header_end = data.find(b"\x00")
    output += data[: header_end + 1]
    i = header_end + 1

    # 각 엔트리 파싱 및 SHA를 .hex()로 변환
    while i < len(data):
        mode_end = data.find(b" ", i)
        name_end = data.find(b"\x00", mode_end)
        mode = data[i:mode_end]
        name = data[mode_end + 1 : name_end]

        sha_start = name_end + 1
        sha_end = sha_start + 20
        sha = data[sha_start:sha_end]
        sha_hex = sha.hex().encode("ascii")

        # 기존 구조 유지
        output += b" " + mode + b" " + name + r"\0".encode() + sha_hex

        i = sha_end

    return bytes(output)


# .git/objects/xx/yyyyyy 에서 Git object 읽기
path = Path(".git", "objects", sys.argv[1][:2], sys.argv[1][2:])
with open(path, "rb") as f:
    compressed = f.read()
    raw = zlib.decompress(compressed)

    result = get_tree_object_content(raw) + b"\n"
    sys.stdout.buffer.write(result)



```


```bash
$ python get_tree_object_content.py bbe651d471cd00fc3add5bdac5c6a41619555f7c
tree 104 100644 file1.txt\00b11cfca50e35a4865e8505f1a108bd23a3f9401 100644 file2.txt\04a8d995103b41f561e547ce3b32e1983d424da22 40000 sub\0e6782efee2e87f68c0d6d87d8bd2a1031dc21f45

$ python get_tree_object_content.py e6782efee2e87f68c0d6d87d8bd2a1031dc21f45
tree 37 100644 file3.txt\05c07b4420198fbb25aabbba53aaa908ca0fb9dd5
```

- 참고로 위 예시 tree 객체의 길이(바이트 수)는
  - 엔트리 1: 6(모드) + 1(공백) + 9(파일명) + 1(`\0`) + 20(해시값) = 37
  - 엔트리 2: 6(모드) + 1(공백) + 9(파일명) + 1(`\0`) + 20(해시값) = 37
  - 엔트리 3: 5(모드) + 1(공백) + 3(디렉토리명) + 1(`\0`) + 20(해시값) = 30
  - 합계: 37 + 37 + 30 = 104

```python
header = b"tree 104\0"
entry1 = b"100644 file1.txt\0" + bytes.fromhex("0b11cfca50e35a4865e8505f1a108bd23a3f9401")
entry2 = b"100644 file2.txt\0" + bytes.fromhex("4a8d995103b41f561e547ce3b32e1983d424da22")
entry3 = b"40000 sub\0" + bytes.fromhex("e6782efee2e87f68c0d6d87d8bd2a1031dc21f45")

tree_object = header + entry1 + entry2 + entry3
hashlib.sha1(tree_object).hexdigest()

# 결과
# 'bbe651d471cd00fc3add5bdac5c6a41619555f7c'

```

## commit 객체

## tag 객체

## git 기본 명령의 내부 구조

- add, commit 등의 명령으로 파일이 등록되는 과정은 내부적으로 다음 과정을 따른다.

| 순번 | git 명령 | 내부 동작 | 설명 |
|-|--|---|------------|
| 1 | git add | blob 객체 생성	| `git hash-object` 명령으로 blob 객체 생성 |
| 2 | - | 인덱스 등록	| `git update-index` 명령으로 인덱스 파일에 스테이징 정보 등록 |
| 3 | git commit | tree 객체 생성	| `git write-tree` 명령으로 인덱스 파일정보를 이용하여 tree 객체 생성 |
| 4 | - | commit 객체 생성 | `git commit-tree` 명령으로 commit 객체 생성 |
| 5 | - | 브랜치 갱신 | `git update-ref` 명령으로 브랜치 정보 갱신 |